In [ ]:
# practice LSTM concept via trials
# 
# https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py

In [5]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

In [6]:
path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

corpus length: 600893


In [7]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i,c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 57


In [11]:
#cut text in semi-redundant sequences
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text)-maxlen, step):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])
print('nb sequences:', len(sentences))

nb sequences: 200285


In [14]:
print('Vectorization')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]]=1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization


In [17]:
#single LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [19]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [20]:
def sample(preds, temperature=1.0):
    #helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [21]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
200192/200285 [============================>.] - ETA: 0s - loss: 1.9905
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "ove should really disenchant the lover w"
ove should really disenchant the lover with and the stands to an in the and a some to superion of the such a some to the supression of the some the some to the say of the some the some to the some the world, and so the supress and not to the some to the despirition of the and the soul and stricity, the despiritionally and so the perions and an in the so the with and and and and the soul with the world to an in the some to the despiritio
----- diversity: 0.5
----- Generating with seed: "ove should really disenchant the lover w"
ove should really disenchant the lover were of despening the sare becond to such and say in the simplys, in the has so has the highes which may and the wish would be distonces, that the or
gower tome the contrary of the soit of the science a provest and i

e up with profound disgust--is of englismuan fundamenatic realifially tuse of a senting inspiring we find may prasert, the opport, the linest communigationflitianves one disturion weaker interprate and taste of their music, and owr highers and antication in any (germans "bailent is man by noith the subtled been dan: prope: i which reashingly dang themself, who subtle plecons-sorase and
roungne: 
greemous clavowing,
not greekings for the
----- diversity: 1.2
----- Generating with seed: "e up with profound disgust--is of englis"
e up with profound disgust--is of english,--suflicic, ableabe tyo
syspet hude. with it byoong reker, so obscloods" accosdarian is to comrous in the boer, losing pressude, the favourable serne, in the higher butly fromve this extetually such opiaiusefur,
whilh
earleth to the rifferer of a showniart no very work only musicable
so moral, to on what extent sufficaty
can
least potsions to presinted, as the lacking things what means thather o
200285/200285 [===========

nowadays in the struggle to the sure the sense of the real intersedity of the light of the same proces and strong and the same be sure and the sense of the sense of the surflection of the surflection of the same the profoundly soul of the religious and the morality of the conscience the profound and the stands of the surflebitions of the stands of the same who have all the sure the finally surrender of the surflec
----- diversity: 0.5
----- Generating with seed: " of what places itself
nowadays in the s"
 of what places itself
nowadays in the strength, the truth it is the human seems to all the badness individual fundamental form of the conservtly and we should my belief in the great stricd who are the will and with a which we can only all the refined the proates the world such a sure of the distress of the and the self-despire of the extented that there is the survanity and finally survers and which the world of the distrust of the sti
----- diversity: 1.0
----- Generating with seed: 

such a pofring, and diler is soulddet soul of them to defering awa.

the
cose the type and "pain
an art enreanic
rewith to
havust useops which immediately
bad does not
prewes.

chieddy evoljtions, the "ord itselves) eaver poetchatormad?-- he to him the pristust of constitutes conjurible, too, is placed--has find 
----- diversity: 1.2
----- Generating with seed: "ary language is called nervous debility "
ary language is called nervous debility st uble to it tong with dod such fiticle danve circuliceless. felt acy can beyo "child amolofispy, frint is tyne, sham-socution cattiqk havy would ais
untrueene on them dogmatulize this very no ilments in pesses not
than he is been
simil aiss"--and delogical adpenoes wherebier and morals--is the charafoulings), only wlan as a
dograised
lord aqual, there honoratibule--and of onepmhins. expressions 
200285/200285 [==============================] - 167s - loss: 1.3977   
Epoch 12/60
200192/200285 [============================>.] - ETA: 0s - loss: 1.3

/Users/grace/workspace/keras/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


...

armbrust (cross-bow). our senses are all the superious and the problem of the reclaise the problem of the proper of the profound the end, in the fact that the same the powerful of the moral instinct and the problem of the pressed to the conscience of the pressermity of the problem of the constraint of the profound and religious and will in the problem of the most subject of the problem of the fact that such a present instrune of the 
----- diversity: 0.5
----- Generating with seed: "to
armbrust (cross-bow). our senses are "
to
armbrust (cross-bow). our senses are in the fact that we not belongs of the most bad and the live in the instinct of himself any community, as inference to the man ambury of the world, which everything itself in the become they in the great art of the strange present that it is that all himself to be companishes of the subject will be something the thing of the art of the
surrender the senses and far and have at least to any longer t
----- diversity: 1.0
----- Ge

accessened inconturied they hars, the saint dehilism at leas where
is a distances: is contemporary its stranged to lokeforics we obeyon,
share once most definitely, sauging science for a se, as sounate systems for order of its turbly metaphysicized and afforre to
the a
secses, "the informal
their
light:--also, those floghtine liketors upon u
----- diversity: 1.2
----- Generating with seed: "
socialistic original. on a more subtle "

socialistic original. on a more subtle to thete ill living, mudglies with which we
tom suiceotcounatter?     imak! where experienciuen bitively
forth keep to haves of whither is be-inherivy: 
,  a prior. teronnesss of shrieop that if rid hylouncileer , 
cree time as-in hims owor also something boacs self
man away has hand to a
 a time in about equally that
in this agreeables (indifessored--never, to place to this finer. tead liffige dr
200285/200285 [==============================] - 162s - loss: 1.3366   
Epoch 31/60
200192/200285 [========================

iently original for an anti-christian philosopher and problem of a standard and also and the self-dangerous and the refined to the extent the conscience of the same and soul, and the developed to the such as the world to the same thing of the self-deed of the such a delight of the standing and such a sureant and also a problem of the such as the such as a complete and self-deed of the such as the self-deed to the pressue to the self-dee
----- diversity: 0.5
----- Generating with seed: "iently original for an anti-christian ph"
iently original for an anti-christian philosophers and another delight of the "such a came is it is the world of the poor and devils in the relations of the most to acts of the influence of the passion of the secret or person of his man is not in the fact to the called to a comes to the end self-partical to the end the self-distress of the section of the such a century that it is also an ancient such a surean of the possibly to german
c
----- diversity: 1.0
-----

circeded stallties, ethics that one in him.! apperiantion of teren--is bott,
and for pleasure, in
the dis, discover his embe as a hundred to generation is yes they what iss from
best or indices lacking and
imperative nove, respecting to comprehe people
has himself disturuison--sick of these bondanily, as i
                     
----- diversity: 1.2
----- Generating with seed: " anything, and owing to an inward self-c"
 anything, and owing to an inward self-calmilal write voices-phjorts to "other?
a truse," in attempt, oolly, from just that one would in
instance vanious proces it
ond no sland-love that cee and first gloom-fitse but power of yling rangen
on one. every faer pacaperiticished--hever his togumant,
the shold and myself one be. how soick and prined
notient, have bey "bying. rapo-carudecar and acceds of cultion!, as agreeng, still passire son
200285/200285 [==============================] - 164s - loss: 1.3288   
Epoch 38/60
200192/200285 [============================>.] - ETA:

KeyboardInterrupt: 